want to roll a FWTD. Will stick a FWTD at both ends of a rigid beam that can rotate freely

### Preamble

In [6]:
import sympy as sym
import sympy.physics.mechanics as me
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import odeint,LSODA,BDF,solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '..\..'))
import sympyTransforms as symt
import custom_plot_objects as cpo
import FwtModels.dof2 as dof2
import FwtModels.AeroModels as ams

me.mechanics_printing()

### Define the Model

#### Create Parameter instance

In [10]:
half = sym.Rational(1,2)

p = dof2.FwtParameters(4)
p.q : sym.Matrix(me.dynamicsymbols(f'q:{4}'))

p.c = dof2.FwtVariable(0.15,'c') # chord
p.k = dof2.FwtVariable(0,'k') # spring constant
p.g  = dof2.FwtVariable(0,'g') # gravity
p.Lambda = dof2.FwtVariable(0,'Lambda') # flare angle
p.rho = dof2.FwtVariable(0,'rho') # density
p.V = dof2.FwtVariable(0,'V') # velocity

p.alpha_r = dof2.FwtVariable(0,'alpha_r') # root AoA

p.a_t  = dof2.FwtVariable(0,'a_t') # C_L of FWT
p.a_w  = dof2.FwtVariable(0,'a_w') # C_L of FWT
 

p.m_t = dof2.FwtVariable(10,'m_t') # wing total mass
p.s_t = dof2.FwtVariable(10,'s_t') # wing total span

p.sigma = dof2.FwtVariable(10,'sigma') # % FWT


p.m_w = p.m_t*(sym.Integer(1)-p.sigma) # mass of inner wing
p.s_w = p.s_t*(sym.Integer(1)-p.sigma) # span of main wing
p.m = p.m_t*p.sigma*half # mass of FWT
p.s = p.s_t*p.sigma*half # span of each FWT

p.I_xx = sym.Rational(1,12)*p.m*p.s**2*1 # inertia of FWT (uniform bar)
p.I_xx_w = sym.Rational(1,12)*p.m_w*p.s_w**2*1 # inertia of FWT (uniform bar)
p.l = p.s*half # location of FWT CoM from Hinge

inner_freq = 2

p.k = (sym.Integer(inner_freq)*2*sym.pi)**2*(p.m_t) # set inner wing Freq to inner_freq

#### Create Transforms

In [11]:
wing_frame = symt.HomogenousTransform().Translate(0,0,p.q[0]).R_x(p.q[1]).R_y(p.alpha_r)

rhs_fwt_frame = wing_frame.Translate(0,p.s_w*half,0).R_x(p.q[2])  # RHS Fwt Frame
lhs_fwt_frame = wing_frame.Translate(0,-p.s_w*half,0).R_x(-p.q[3])  # LHS Fwt Frame

rhs_fwt_com_frame = rhs_fwt_frame.Translate(0,p.l,0)
lhs_fwt_com_frame = lhs_fwt_frame.Translate(0,-p.l,0)

rhs_rot = sym.Matrix([p.q[1]+p.q[2],p.alpha_r,0])
lhs_rot = sym.Matrix([p.q[1]-p.q[2],p.alpha_r,0])
wing_rot = sym.Matrix([p.q[1],p.alpha_r,0])


M_fwt = dof2.MassMatrix(p.m,I_xx = p.I_xx)
M_w = dof2.MassMatrix(p.m_w,I_xx = p.I_xx_w)

m_rhs = dof2.RigidElement(rhs_fwt_com_frame,rhs_rot,M_fwt)
m_lhs = dof2.RigidElement(lhs_fwt_com_frame,lhs_rot,M_fwt)
m_wing = dof2.RigidElement(wing_frame,wing_rot,M_w)

#rhs_AeroForces = ams.AeroModelv1(p,fwt_frame,at_mode=1)
rhs_GravityForces = ams.GravityModel(p,rhs_fwt_com_frame,sym.Matrix([0,0,p.g*p.m]))
#lhs_AeroForces = ams.AeroModelv1(p,fwt_frame,at_mode=1)
lhs_GravityForces = ams.GravityModel(p,lhs_fwt_com_frame,sym.Matrix([0,0,p.g*p.m]))





In [12]:
m_lhs.Jacobian(p.q)

⎡                                                                             
⎢0                                     0                                     0
⎢                                                                             
⎢                                                                             
⎢         ⎛sin(q₁)⋅cos(q₃)   sin(q₃)⋅cos(αᵣ)⋅cos(q₁)⎞      ⎛σ   1⎞            
⎢0   sₜ⋅σ⋅⎜─────────────── - ───────────────────────⎟ - sₜ⋅⎜─ - ─⎟⋅sin(q₁)   0
⎢         ⎝       4                     4           ⎠      ⎝2   2⎠            
⎢                                                                             
⎢        ⎛  sin(q₁)⋅sin(q₃)⋅cos(αᵣ)   cos(q₁)⋅cos(q₃)⎞      ⎛σ   1⎞           
⎢1  sₜ⋅σ⋅⎜- ─────────────────────── - ───────────────⎟ + sₜ⋅⎜─ - ─⎟⋅cos(q₁)  0
⎢        ⎝             4                     4       ⎠      ⎝2   2⎠           
⎢                                                                             
⎢0                                     1            